In [271]:
import json
import requests
import pandas as pd

url='https://lefooding.com/fr/search?type=restaurant&filters%5Bgeo%5D%5Bcity%5D%5B%5D=526ba3f07f55b7f39a007e28&page=1&sort=&per_page=50&total_pages=200'
r = requests.get(url)
print(url, r.status_code)
#r.json(),r.content()

data = r.json()

with open('data_paris.json', 'w') as outfile:
    json.dump(data, outfile, indent = 4)

https://lefooding.com/fr/search?type=restaurant&filters%5Bgeo%5D%5Bcity%5D%5B%5D=526ba3f07f55b7f39a007e28&page=1&sort=&per_page=50&total_pages=200 200


In [272]:
restau_name = []
restau_address = []
restau_code_postal = []
restau_type = []
restau_phone = []
restau_price = []
restau_latitude = []
restau_longitude = []

for elem in data['results']:
    restau_name.append(elem['name'])
    restau_type.append(elem['cravings'])    
    restau_phone.append(elem['phone_num'])
    restau_price.append(elem['price'])
    restau_address.append(elem['address']['street_name_1'])
    restau_code_postal.append(str(elem['address']['postal_code']))
    restau_latitude.append(elem['address']['position'][0])
    restau_longitude.append(elem['address']['position'][1])

df = pd.DataFrame({'Nom': restau_name, 'Rue': restau_address, 'Code postal': restau_code_postal,
                   'Type': restau_type, 'Numero tel': restau_phone , 'Prix': restau_price, 'Latitude': restau_latitude, 'Longitude' : restau_longitude})
df.to_csv(r'data_paris.csv', sep = ',', index = True, encoding = 'utf-8')

df = pd.read_csv('data_paris.csv',sep=',',encoding='utf-8', index_col=['Nom']).sort_index()

del df['Unnamed: 0']

df

,Rue,Code postal,Type,Numero tel,Prix,Latitude,Longitude
Nom,,,,,,,
Autour du Yangtse,"12, rue du Helder",75009,['Chinois'],+33 1 53 34 05 78,['Moins de 15 €'],48.872033,2.335064
Aux Prés,"27, rue du Dragon",75006,['Néobistrot'],+33 1 45 48 29 68,['Plus de 51 €'],48.852724,2.330512
Banh Mi,"81, rue de Turbigo",75003,"['Asiatique', 'Vietnamien', 'Snack / Tartes / ...",+33 1 48 04 98 90,['Moins de 15 €'],48.866667,2.360680
Bar à Momos,"218, rue du Faubourg Saint-Antoine",75012,"['Lèche-doigts', 'Tibétain']",+33 1 43 72 89 82,['Moins de 15 €'],48.849565,2.387085
Basique,"164, av. Ledru-Rollin",75011,['Tapas'],+33 1 40 24 20 48,['De 16 à 35 €'],48.856626,2.379403
Bontemps,"57, rue de Bretagne",75003,"['Salon de thé / Pâtisserie', 'Snack / Tartes ...",+33 1 42 74 10 68,['De 16 à 35 €'],48.863817,2.360690
Cantina,"11, rue Marie-et-Louise",75010,['Néobistrot'],Pas de téléphone.,['De 16 à 35 €'],48.871880,2.366474
Chez Alain Miam Miam,"26, rue Charlot",75003,"['Biocool', 'Crêperie', 'Sandwichs / Bagels', ...",+33 9 86 17 28 00,['Moins de 15 €'],48.862260,2.362022
Chinaski,"46, rue Daubenton",75005,"['Café / Coffee Shop', ""Cuisine d'auteur"", 'Né...",+33 1 73 74 74 06,['De 16 à 35 €'],48.840358,2.350236


# Fonctions filtre

In [273]:
#fonctions
#permet de récupérer les restau par type
def restauParType(type):    
    
    restauParType = df[df['Type'].str.contains(type)]
    return restauParType;

#restauParType("Cuisine d'auteur")

def restauParCP(cp):    
    
    restauParCP = df[df['Code postal'] == cp]
    return restauParCP;

#restauParCP(75007)

def restau(nom):    
            
    restau = df.loc[nom]
    return restau;

restau('Plume')
#restau('Plume').Rue



Rue                         24, rue Pierre-Leroux
Code postal                                 75007
Type           ["Cuisine d'auteur", 'Néobistrot']
Numero tel                      +33 1 43 06 79 85
Prix                             ['De 36 à 50 €']
Latitude                                  48.8488
Longitude                                  2.3192
Name: Plume, dtype: object

# MAP avec folium

In [274]:

# si vous n'avez pas installer folium -> !pip install folium

#affiche tous les restau sur la map
import folium

def map_all(): 
    map_restau = folium.Map(
    location=[48.8666667, 2.33333333],
    zoom_start=13)
    
    tooltip = 'Detail'
      
    for i in df.index:        
        folium.Marker([df['Latitude'].loc[i], df['Longitude'].loc[i]], popup=i, tooltip=tooltip).add_to(map_restau)
    return map_restau

#affiche seulement les restau selon le type entré n'appelle pas la fonction filter car plus long... et c'est plus simple

def map_type(type):
    map_type = folium.Map(
    location=[48.8666667, 2.33333333],
    zoom_start=13)
    
    tooltip = 'Detail'
      
    for i in df.index:        
        if df['Type'].str.contains(type).loc[i]:
            folium.Marker([df['Latitude'].loc[i], df['Longitude'].loc[i]], popup=i, tooltip=tooltip).add_to(map_type)
    return map_type


#affiche les restau par CP

def map_cp(cp):
    map_cp = folium.Map(
    location=[48.8666667, 2.33333333],
    zoom_start=13)
    
    tooltip = 'Detail'
      
    for i in df.index:        
        if df['Code postal'].loc[i] == cp:
            folium.Marker([df['Latitude'].loc[i], df['Longitude'].loc[i]], popup=i, tooltip=tooltip).add_to(map_cp)
    return map_cp


map_cp(75001)



Entrées / saisies